In [3]:
import pandas as pd
import numpy as np
import math as m

In [27]:
data=pd.read_csv('stock/stock_clean.csv')
data_T=pd.DataFrame(data, columns= ['Object','Property','Value','Source'])
data_T

,Object,Property,Value,Source
0,sial,Change %,1.170000e+00,advfn
1,sial,Last trading price,7.424000e+01,advfn
2,sial,Open price,7.354000e+01,advfn
3,sial,Change $,8.600000e-01,advfn
4,sial,Volume,6.746630e+05,advfn
...,...,...,...,...
56987,shld,Today's low,6.965000e+01,zacks
56988,shld,52wk High,9.479000e+01,zacks
56989,shld,52wk Low,5.921000e+01,zacks
56990,shld,Market cap,7.750000e+09,zacks


In [28]:
data_T['TSInitial']=0.8
data_T['C_v']=0.0
data_T.head(5)

,Object,Property,Value,Source,TSInitial,C_v
0,sial,Change %,1.17,advfn,0.8,0.0
1,sial,Last trading price,74.24,advfn,0.8,0.0
2,sial,Open price,73.54,advfn,0.8,0.0
3,sial,Change $,0.86,advfn,0.8,0.0
4,sial,Volume,674663.00,advfn,0.8,0.0


In [29]:
def word2vec(word):
    from collections import Counter
    from math import sqrt

    # count the characters in word
    cw = Counter(word)

    # precomputes a set of the different characters
    sw = set(cw)
    # precomputes the "length" of the word vector
    lw = sqrt(sum(c*c for c in cw.values()))

    # return a tuple
    return cw, sw, lw

def cosdis(a, b):
    v1 = word2vec(a)
    v2 = word2vec(b)
    # which characters are common to the two words?
    common = v1[1].intersection(v2[1])
    # by definition of cosine distance we have
    return sum(v1[0][ch]*v2[0][ch] for ch in common)/v1[2]/v2[2]

In [30]:
# Definition de la matice de similarite
def similarity(dataframe):
    dict_sim = {}
    for key, df in dataframe.groupby(by=['Object','Property']):
        Values = df['Value'].unique()
        row = key[0]+key[1]
        for  i in range(len(Values)):
            w1 = Values[i]
            for  u in range(len(Values)):
                w2 = Values[u]
                sim = 1
                if w1!=w2:
                    t = abs(w1-w2)
                    sim = 1/t
                dict_sim[row+str(w1)+str(w2)] = sim
                dict_sim[row+str(w2)+str(w1)] = sim
    return dict_sim


# def similarity(dataframe):
#     dict_sim = {}
#     for key, df in dataframe.groupby(by=['Object','Property']):
#         Values = df['Value'].unique()
#         row = key[0]+key[1]
#         for  i in range(len(Values)):
#             w1 = Values[i]
#             for  u in range(len(Values)):
#                 w2 = Values[u]
#                 sim = 1
#                 if w1!=w2:
#                     sim = cosdis(w1,w2)
#                 dict_sim[row+str(w1)+str(w2)] = sim
#                 dict_sim[row+str(w2)+str(w1)] = sim
#     return dict_sim

In [31]:
import os
import psutil



def get_process_memory():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss

In [32]:
import math
def Ts(df):
    ts_list = df['TSInitial'].to_list()
    return sum([-math.log(1-ts) for ts in ts_list])
def cos_sim(a,b):
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))

In [33]:
import time as t
from natsort import index_natsorted, order_by_index
def TruthFinder_seq(data_T):
    mem_before = get_process_memory()
    ni=5
    trustold = 0.00001
    start = t.time()
    nb_iter_do = 0
    rho=0.7
    lam=0.5
    dict_sim=similarity(data_T)
    for i in range(ni):
        df = data_T[['Source', 'TSInitial']].drop_duplicates().reindex(index=order_by_index(data_T[['Source', 'TSInitial']].drop_duplicates().index, index_natsorted(data_T[['Source', 'TSInitial']].drop_duplicates()['Source'], reverse=False)))
        ts1=df['TSInitial']
        grouped_df = data_T.groupby(["Object", "Property","Value"])
        for key,item in grouped_df:
            sum_ = Ts(item)
            data_T.loc[((data_T["Object"] == key[0]) & (data_T["Property"]==key[1]) & (data_T["Value"]==key[2])), "C_v"]=sum_
        grouped_df=data_T.groupby(["Object", "Property","Value"])
        for key ,item in grouped_df:
            cv=data_T.loc[((data_T["Object"] == key[0]) & (data_T["Property"]==key[1]) & (data_T["Value"]==key[2])), "C_v"].values[0]
#             print(cv)

            data_d=data_T.loc[((data_T["Object"] == key[0]) & (data_T["Property"]==key[1]) & (data_T["Value"]!=key[2])), ["Value","C_v"]]
            for row in data_d.values:
                Key_sim=key[0]+key[1]+str(key[2])+str((row[0]))
#                 Key_sim=key[0]+key[1]+str(key[2])+str(int(row[0]))
                cv+=rho*(row[1]*dict_sim[Key_sim])
            cv=1/(1+np.exp(-lam*cv))
        #     print(cv)
            data_T.loc[((data_T["Object"] == key[0]) & (data_T["Property"]==key[1]) & (data_T["Value"]==key[2])), "C_v"]=cv
        groupe_ts=data_T.groupby(['Source'])
        for key,item in groupe_ts:
            ts_=item['C_v'].mean()
            if ts_ >=1:
                ts_ = 1-0.0001
            elif ts_ <= 0:
                ts_ = 0 + 0.0001
            data_T.loc[((data_T["Source"] == key)), "TSInitial"]=ts_
        df = data_T[['Source', 'TSInitial']].drop_duplicates().reindex(index=order_by_index(data_T[['Source', 'TSInitial']].drop_duplicates().index, index_natsorted(data_T[['Source', 'TSInitial']].drop_duplicates()['Source'], reverse=False)))
        ts2=df['TSInitial']
        print('Itération : ',i+1)
        print(1-abs(cos_sim(ts1,ts2)))
        nb_iter_do = i+1
        if 1-abs(cos_sim(ts1,ts2)) < trustold:
            break
    end = t.time()
    mem_after = get_process_memory()

    return data_T,end-start,nb_iter_do,mem_after,mem_before,mem_after-mem_before



In [34]:
result_=TruthFinder_seq(data_T)

Itération :  1
0.00019296684140013287
Itération :  2
4.1943600496252564e-05
Itération :  3
3.680384619508814e-06


In [35]:
result_

(      Object            Property         Value Source  TSInitial       C_v
 0       sial            Change %  1.170000e+00  advfn   0.999147  1.000000
 1       sial  Last trading price  7.424000e+01  advfn   0.999147  1.000000
 2       sial          Open price  7.354000e+01  advfn   0.999147  1.000000
 3       sial            Change $  8.600000e-01  advfn   0.999147  1.000000
 4       sial              Volume  6.746630e+05  advfn   0.999147  0.999258
 ...      ...                 ...           ...    ...        ...       ...
 56987   shld         Today's low  6.965000e+01  zacks   0.991433  1.000000
 56988   shld           52wk High  9.479000e+01  zacks   0.991433  1.000000
 56989   shld            52wk Low  5.921000e+01  zacks   0.991433  1.000000
 56990   shld          Market cap  7.750000e+09  zacks   0.991433  0.899305
 56991   shld                 EPS  1.200000e-01  zacks   0.991433  1.000000
 
 [56992 rows x 6 columns],
 853.9828717708588,
 3,
 149061632,
 138903552,
 10158080)

In [37]:
# def res(result):
#     gr=result.groupby(["Object", "Property"])
#     data_out={'Object':[],'Property':[],'Value':[]}
#     for key,item in gr:
# #         print(item.values)
#         cv_max=0
#         value_pred=None
#         for line in item.values:
# #             print(line[5])
#             if cv_max<line[5]:
#                 cv_max=line[5]
#                 value_pred=line[2]
#         data_out['Object'].append(key[0])
#         data_out['Property'].append(key[1])
#         data_out['Value'].append(value_pred)
#     return pd.DataFrame(data_out)
        
def res(result):
    gr=result.groupby(["Object", "Property"])
    data_out={'Object':[],'Property':[],'Value':[]}
    for key,item in gr:
#         print(item.values)
        cv_max=0
        value_pred=None
        for line in item.values:
#             print(line[5])
            if cv_max<line[5]:
                cv_max=line[5]
                if value_pred==None:
                    value_pred=line[2]
            else:
                value_pred=max(value_pred,line[2])
        data_out['Object'].append(key[0])
        data_out['Property'].append(key[1])
        data_out['Value'].append(value_pred)
    return pd.DataFrame(data_out)

In [38]:
res_=res(result_[0])
len(res_)

548

In [39]:
list_ = res_.to_numpy()

In [40]:
data_truth=pd.read_csv('flight/flight_truth.csv')
data_truth_=pd.DataFrame(data_truth, columns= ['Object','Property','Value'])
data_truth

,Object,Property,Value
0,AA-1221-MCO-ORD,Scheduled departure,2011-12-01 8:00 PM
1,AA-1221-MCO-ORD,Actual departure,2011-12-01 8:23 PM
2,AA-1221-MCO-ORD,Departure gate,17
3,AA-1221-MCO-ORD,Scheduled arrival,2011-12-01 9:45 PM
4,AA-1221-MCO-ORD,Actual arrival,2011-12-01 9:53 PM
...,...,...,...
543,CO-89-IAH-EWR,Actual arrival,2011-12-01 10:27 AM
544,CO-1090-BOS-IAH,Scheduled departure,2011-12-01 9:45 AM
545,CO-1090-BOS-IAH,Actual departure,2011-12-01 10:19 AM
546,CO-1090-BOS-IAH,Scheduled arrival,2011-12-01 1:16 PM


In [41]:
len(data_truth_)

548

In [42]:
list__=data_T.to_numpy()
# list__

In [43]:
list_1=data_truth_.to_numpy()


In [44]:
# def function(list_A,list_B):
#     binary=[]
#     search_list = [tuple(li[:-3]) for li in list_B]
#     for ser_item in search_list:
#         if ser_item in list_A:
#             binary.append(1)
#         else:
#             binary.append(0)
#     return binary

In [45]:
def function(list_A,list_B):
    binary=[]
    search_list = [tuple(li[:-3]) for li in list_B]
    list_A = [tuple(li) for li in list_A]
    for ser_item in search_list:
        if ser_item in list_A:
            binary.append(1)
        else:
            binary.append(0)
    return binary

In [46]:
res = function(list_, list__)
res_truth=function(list_1, list__)
# res
# res_truth

In [47]:
# !pip install scikit-learn
import sklearn

In [48]:
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [49]:
precision_score(res_truth,res)

0.8222988505747126

In [50]:
accuracy_score(res_truth,res)

0.7431307718618173

In [51]:
recall_score(res_truth,res)

0.7073363654340518

In [52]:
f1_score(res_truth,res)

0.7604975018603166